# Try classification by feature selection

In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()

from google.colab import drive
drive.mount('/content/drive')

sys.path.append("/content/drive/MyDrive/GSOC-NMR-project/Work/Notebooks")
from auxillary_functions import *
from polynomial_featextract import poly_featextract

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import raw data and params.txt file
datadir_path = "/content/drive/MyDrive/GSOC-NMR-project/Work/Data/2021-06-21_classify_datagen_all_funcs"

rawdata = load_data(datadir_path)
params = load_params(datadir_path)
ker_integrals = load_wlist(datadir_path) # load wlist.txt file

# Stencil type : {'0' : 'Gaussian', '1' : 'Power Law', '2' : 'RKKY'}

Finished loading rawdata into numpy array
Finsihed loading parameters file
finished loading kernel-integrals file.


In [3]:
print(rawdata.shape)
offset = 150
shifted_data, center = get_window(rawdata,2/3,width=offset)
print("The Echo pulse occurs at timestep:",center)

# Rescaled data
rscl_data = shifted_data / np.max(shifted_data,axis=1,keepdims=True)

(10500, 943)
The Echo pulse occurs at timestep: 628


# Classification

In [32]:
!pip uninstall scikit-learn -y

!pip install -U scikit-learn

Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached https://files.pythonhosted.org/packages/a8/eb/a48f25c967526b66d5f1fa7a984594f0bf0a5afafa94a8c4dbc317744620/scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl


In [4]:
import sklearn 
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier

In [ ]:
print(sklearn.__version__)

In [5]:
X_train, X_test, y_train, y_test = tts(rscl_data, params["stencil_type"].values, test_size=0.3, random_state=42)

In [22]:
# ?RandomForestClassifier
model = RandomForestClassifier(n_estimators= 40, random_state=0,
                               n_jobs=-1, oob_score=True)

In [23]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
                       oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

In [24]:
y_pred = model.predict(X_test)

In [25]:
model.score(X_test, y_test)

0.9707936507936508

In [11]:
sklearn.metrics.cohen_kappa_score(y_test, y_pred)

0.9385852062817653

In [12]:
sklearn.metrics.matthews_corrcoef(y_test, y_pred)

0.939079703085025

0.22.2.post1


In [28]:
from sklearn.model_selection import GridSearchCV, train_test_split, HalvingGridSearchCV
from sklearn.pipeline import Pipeline

ImportError: ignored

In [27]:
pipe = Pipeline([
                 ('model',RandomForestClassifier(random_state=0,
                                                 n_jobs=True, oob_score=True ))
])